# **DEEP LEARNING**

**UNIVERSIDAD MILITAR NUEVA GRANADA**

**Proyecto 1**

Alejandro Mejia Rojas

## Importación de Librerías


In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Depurar DataSet

## Lectura del dataset

In [2]:
# Cargar el archivo
file_path = 'attachments/Conjunto_servidores_p_blicos_20240819.csv'
df = pd.read_csv(file_path, dtype={'Columna_8': str, 'Columna_9': str}, low_memory=False)

## Filtrar las columnas relevantes 

In [3]:
df = df[['Meses de Experiencia Público', 'Meses de Experiencia Privado',
        'Nivel Educativo', 'Tipo de Nombramiento', 'Nivel Jerarquico Empleo',
        'Dependencia Empleo Actual', 'Asignación Básica Salarial']]

## Cambiar los valores Nulos dentro de las columnas por 0 
## Eliminar las filas que tengan Nulos 

In [4]:
# Convertir las columnas numéricas y rellenar nulos
df.loc[:, 'Meses de Experiencia Público'] = pd.to_numeric(df['Meses de Experiencia Público'], errors='coerce').fillna(0)
df.loc[:, 'Meses de Experiencia Privado'] = pd.to_numeric(df['Meses de Experiencia Privado'], errors='coerce').fillna(0)

df = df.dropna(subset=['Nivel Educativo', 'Nivel Jerarquico Empleo', 'Dependencia Empleo Actual', 'Asignación Básica Salarial'])

# Convertir la columna 'Asignación Básica Salarial' a valores numéricos y escalarla a millones
df['Asignación Básica Salarial'] = df['Asignación Básica Salarial'].replace({'$': '', ',': ''}, regex=True).astype(float)
df['Asignación Básica Salarial'] = df['Asignación Básica Salarial'] / 1_000_000

# One-Hot Encoding para las columnas categóricas

In [5]:
df = pd.get_dummies(df, columns=['Nivel Educativo', 'Tipo de Nombramiento', 'Nivel Jerarquico Empleo', 'Dependencia Empleo Actual'])

df.head()

,Meses de Experiencia Público,Meses de Experiencia Privado,Asignación Básica Salarial,Nivel Educativo_BASICA PRIMARIA,Nivel Educativo_BASICA SECUNDARIA,Nivel Educativo_EDUCACION MEDIA,Nivel Educativo_POSTGRADO,Nivel Educativo_PREGRADO,Tipo de Nombramiento_CONTRATO LABORAL,Tipo de Nombramiento_DESIGNACIÓN,...,Dependencia Empleo Actual_ÁREA TECNICO OPERATIVA,Dependencia Empleo Actual_ÁREA TECNOLOGÍAS DE LA INFORMACIÓN Y TELECOMUNICACIONES TICS,Dependencia Empleo Actual_ÁREA TÉCNICA,Dependencia Empleo Actual_ÁREA TÉCNICA OPERATIVA,Dependencia Empleo Actual_Área Administrativa,Dependencia Empleo Actual_Área Comercial,Dependencia Empleo Actual_Área Misional,Dependencia Empleo Actual_Área Operativa,Dependencia Empleo Actual_ÓRGANOS DE ASESORÍA Y COORDINACIÓN,Dependencia Empleo Actual_Órganos de Asesoría y Coordinación
2,143.0,0.0,5.458553,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,421.0,337.0,3.055244,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,215.0,96.0,1.747269,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
7,497.0,0.0,5.691032,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,176.0,0.0,11.321727,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Definir conjuntos entrenamiento y prueba

In [6]:
# Definir características (X) y etiqueta (y)
X = df.drop(columns=['Asignación Básica Salarial'])
y = df['Asignación Básica Salarial']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir todas las columnas de X_train y X_test a tipo int
X_train = X_train.astype(int)
X_test = X_test.astype(int)

X_train.head()

,Meses de Experiencia Público,Meses de Experiencia Privado,Nivel Educativo_BASICA PRIMARIA,Nivel Educativo_BASICA SECUNDARIA,Nivel Educativo_EDUCACION MEDIA,Nivel Educativo_POSTGRADO,Nivel Educativo_PREGRADO,Tipo de Nombramiento_CONTRATO LABORAL,Tipo de Nombramiento_DESIGNACIÓN,Tipo de Nombramiento_ELECCIÓN,...,Dependencia Empleo Actual_ÁREA TECNICO OPERATIVA,Dependencia Empleo Actual_ÁREA TECNOLOGÍAS DE LA INFORMACIÓN Y TELECOMUNICACIONES TICS,Dependencia Empleo Actual_ÁREA TÉCNICA,Dependencia Empleo Actual_ÁREA TÉCNICA OPERATIVA,Dependencia Empleo Actual_Área Administrativa,Dependencia Empleo Actual_Área Comercial,Dependencia Empleo Actual_Área Misional,Dependencia Empleo Actual_Área Operativa,Dependencia Empleo Actual_ÓRGANOS DE ASESORÍA Y COORDINACIÓN,Dependencia Empleo Actual_Órganos de Asesoría y Coordinación
63670,281,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
151691,81,83,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35470,251,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
65736,106,24,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98003,186,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Modelo

In [7]:
# Definir el modelo 
model = Sequential()

# Agregar capas 
model.add(Dense(256, input_shape=(X_train.shape[1],)))  # capa de entrada
model.add(Dense(128))  # capa oculta
model.add(Dense(64))   # Otra capa oculta
model.add(Dense(1))    # Capa de salida


c:\Users\aleja\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## hyperparametros

In [8]:
# Compilar el modelo con diferentes hiperparámetros
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=["mae"])

# Resumen del modelo
model.summary()

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, verbose=1)

# Evaluar el modelo en el conjunto de prueba
test_loss, test_mse = model.evaluate(X_test, y_test)

print(f"Loss en el conjunto de prueba: {test_loss}, MSE en el conjunto de prueba: {test_mse}")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │     2,232,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,274,049 (8.67 MB)

 Trainable params: 2,274,049 (8.67 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - loss: 10.8586 - mae: 1.6236 - val_loss: 1.5470 - val_mae: 0.8054
Epoch 2/50
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 25s 14ms/step - loss: 1.7480 - mae: 0.8608 - val_loss: 1.5861 - val_mae: 0.8161
Epoch 3/50
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - loss: 1.7100 - mae: 0.8505 - val_loss: 2.5666 - val_mae: 1.1538
Epoch 4/50
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - loss: 1.7121 - mae: 0.8414 - val_loss: 1.6300 - val_mae: 0.8755
Epoch 5/50
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - loss: 1.5923 - mae: 0.8199 - val_loss: 1.7917 - val_mae: 0.9617
Epoch 6/50
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - loss: 1.7053 - mae: 0.8328 - val_loss: 1.5793 - val_mae: 0.8617
Epoch 7/50
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - loss: 1.6165 - mae: 0.8207 - val_loss: 1.5082 - val_mae: 0.7981
Epoch 8/50
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - loss: 1.5539 - mae: 0.8084 - val_loss: 1.5421 - val_mae: 0.8150
Epoch 9/50
1775/1775 ━━